# 学习PyTorch-Day5

训练模型是一个迭代过程：在每次迭代中，模型都会对输出进行猜测，计算其猜测的误差（损失），收集误差相对于其参数的导数，并使用梯度下降优化这些参数

## 例

In [56]:
import torch

device = torch.device(
    "mps"
    if torch.backends.mps.is_available()
    else "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(device)

mps


In [57]:
from torch.utils.data import DataLoader
import torch.nn as nn
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=False,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=False,
    transform=ToTensor()
)

train_dataloader = DataLoader(dataset=training_data, batch_size=16, shuffle=True)

test_dataloader = DataLoader(dataset=test_data, batch_size=16, shuffle=True)

class NeuralNetwork(nn.Module):
    def __init__(self) -> None:
        super(NeuralNetwork, self).__init__()
        self.flatten  = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 10),
        )
        
    def forward(self, x):
        x = self.flatten(x)
        x = self.linear_relu_stack(x)
        return x

model = NeuralNetwork()
model.to(device)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

## 超参数

- 超参数是一堆可以控制整个优化过程的可调整参数
- 不同的超参数可以对训练过程及模型收敛造成影响

1. __Number of Epochs__：在数据集上迭代的次数
2. __Batch Size__：参数更新之前通过网络传播的数据样本的数量
3. __Learning Rate__：每个 __batch/epoch__ 更新模型参数的数量。较小的学习率会导致学习速度较慢，而较大的学习率可能会导致训练期间出现不可预测的行为

In [58]:
epochs = 10
batch_size = 16
lr = 1e-3

## 优化循环

### 损失函数

- 回归任务 —— __nn.MSELoss__ (Mean Square Error)
- 分类任务 —— __nn.NLLLoss__ (Negative Log Likelihood)
- __nn.CrossEntropyLoss__ 包括了 __nn.LogSoftMax__ 和 __nn.NLLLoss__

In [59]:
# 初始化损失函数
loss_fn = nn.CrossEntropyLoss()

### 优化器

- 所有的优化逻辑都被封装在 __optimizer__ 对象中

#### 初始化优化器

- 初始化需要传入的参数
1. 整个模型的网络参数 __model.parameters( )__
2. 学习率 __learning_rate__

In [60]:
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

#### 优化过程

1. 调用 __optimizer.zero_grad( )__ 来重置所有模型参数的梯度
2. 通过调用 __loss.backward( )__ 反向传播预测损失
3. 一旦我们得到了梯度，我们就调用 __optimizer.step( )__ 来调整参数

## 具体实现

In [61]:
def train_loop(dataloader, model, loss_fn, optimizer):
    """
        训练过程
    Args:
        dataloader (_type_): _description_
        model (_type_): _description_
        loss_fn (_type_): _description_
        optimizer (_type_): _description_
    """
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # 预测
        pred = model(X.to(device))
        # 基于预测结果和标签计算 loss
        loss = loss_fn(pred, y.to(device))
        
        # 反向传播
        loss.backward()
        
        # 优化参数
        optimizer.step()
        
        # 重置梯度
        optimizer.zero_grad()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [62]:
def test_loop(dataloader, model, loss_fn):
    """
        测试过程
    Args:
        dataloader (_type_): _description_
        model (_type_): _description_
        loss_fn (_type_): _description_
    """
    size = len(dataloader.dataset)
    print("size={}".format(size))
    model.eval()
    num_batches = len(dataloader)
    print("num_batches={}".format(num_batches))
    
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X.to(device))
            test_loss += loss_fn(pred, y.to(device)).item()
            correct += (pred.argmax(1) == y.to(device)).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size    
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [63]:
import time

t1 = time.time()

for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    train_loop(dataloader=train_dataloader, model=model, loss_fn=loss_fn, optimizer=optimizer)
    test_loop(dataloader=test_dataloader, model=model, loss_fn=loss_fn)
    
t2 = time.time()

print("All Done in {} secs !!!".format(t2 - t1))

Epoch 1
-------------------------------
loss: 2.313297  [   16/60000]


loss: 2.293334  [ 1616/60000]
loss: 2.273076  [ 3216/60000]
loss: 2.272763  [ 4816/60000]
loss: 2.246865  [ 6416/60000]
loss: 2.231766  [ 8016/60000]
loss: 2.246040  [ 9616/60000]
loss: 2.241057  [11216/60000]
loss: 2.211491  [12816/60000]
loss: 2.189331  [14416/60000]
loss: 2.168194  [16016/60000]
loss: 2.086901  [17616/60000]
loss: 2.108777  [19216/60000]
loss: 2.118653  [20816/60000]
loss: 2.128221  [22416/60000]
loss: 2.057017  [24016/60000]
loss: 1.918358  [25616/60000]
loss: 1.970721  [27216/60000]
loss: 1.949620  [28816/60000]
loss: 1.971829  [30416/60000]
loss: 1.893253  [32016/60000]
loss: 1.800910  [33616/60000]
loss: 1.838858  [35216/60000]
loss: 1.568265  [36816/60000]
loss: 1.627336  [38416/60000]
loss: 1.675721  [40016/60000]
loss: 1.756574  [41616/60000]
loss: 1.570633  [43216/60000]
loss: 1.742628  [44816/60000]
loss: 1.485454  [46416/60000]
loss: 1.246074  [48016/60000]
loss: 1.412761  [49616/60000]
loss: 1.393174  [51216/60000]
loss: 1.289234  [52816/60000]
loss: 1.26